<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_deepmatch_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
# ! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2022-09-29 18:40:18--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  6.37MB/s    in 0.9s    

2022-09-29 18:40:20 (6.37 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-29 18:40:20--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [ ]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

# Data 

In [ ]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
data.user_id.nunique(), data.movie_id.nunique(), data.head(2)

(46, 2175,    user_id  movie_id  rating  timestamp  \
 0        1      1193       5  978300760   
 1        1       661       3  978302109   
 
                                     title     genres gender  age  occupation  \
 0  One Flew Over the Cuckoo's Nest (1975)      Drama      F    1          10   
 1        James and the Giant Peach (1996)  Animation      F    1          10   
 
      zip  
 0  48067  
 1  48067  )

# Feature Engineering

In [ ]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
negsample = 1


In [ ]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
feature_max_idx 

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [ ]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18
2,1,483,3,978301968,My Fair Lady (1964),12,1,1,9,18
3,1,1881,4,978300275,Erin Brockovich (2000),8,1,1,9,18
4,1,1306,5,978824291,"Bug's Life, A (1998)",3,1,1,9,18


In [ ]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [ ]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [ ]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)
len(train_set), len(test_set)

100%|██████████| 46/46 [00:00<00:00, 145.90it/s]

8 8


(19816, 46)

In [ ]:
data.loc[2]

user_id                         1
movie_id                      483
rating                          3
timestamp               978301968
title         My Fair Lady (1964)
genres                         12
gender                          1
age                             1
occupation                      9
zip                            18
Name: 2, dtype: object

In [ ]:
print(train_set[0])

# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - constant
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len,  - 50
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating

(42, 287, 1, [916, 852, 1505, 1197, 1982, 604, 1957, 585, 659, 655, 1921, 629, 686, 715, 152, 663, 1438, 649, 1160, 648, 354, 601, 903, 958, 2033, 25, 1650, 1121, 848, 647, 320, 1083], 32, [8, 1, 1, 4, 1, 4, 1, 5, 1, 15, 8, 5, 1, 5, 1, 1, 1, 1, 2, 15, 1, 8, 13, 5, 8, 2, 8, 1, 5, 1, 8, 5], 1, 4)


In [ ]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [ ]:
train_model_input

{'user_id': array([42, 15, 10, ..., 42, 40, 34]),
 'movie_id': array([ 287,  708,  623, ..., 1017,  778,  504]),
 'hist_movie_id': array([[ 916,  852, 1505, ...,    0,    0,    0],
        [ 944, 1591, 1544, ...,    0,    0,    0],
        [ 592,  343, 1859, ..., 1940, 2038, 2036],
        ...,
        [ 105, 1775, 1956, ..., 1813,  313, 1357],
        [1304, 1712,  587, ...,  926, 1656, 2027],
        [1002,  903, 1425, ...,  611, 1135,  232]], dtype=int32),
 'hist_genres': array([[ 8,  1,  1, ...,  0,  0,  0],
        [ 8,  5,  3, ...,  0,  0,  0],
        [ 6, 11,  5, ...,  5,  8, 18],
        ...,
        [ 8,  8,  5, ...,  8,  1,  5],
        [ 5,  5,  5, ...,  8,  8,  8],
        [ 8, 13,  5, ...,  1,  1,  1]], dtype=int32),
 'hist_len': array([32, 29, 50, ..., 50, 50, 50]),
 'genres': array([ 1,  8,  6, ...,  5,  8, 12]),
 'gender': array([2, 2, 2, ..., 2, 1, 2]),
 'age': array([2, 3, 5, ..., 2, 3, 4]),
 'occupation': array([ 5,  5,  4, ...,  5, 17,  8]),
 'zip': array([37, 36, 

# Aggregate Feature and create Embeddings

In [ ]:
embedding_dim = 32

In [ ]:
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

In [ ]:
item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)


In [ ]:
len(sampler_config.item_count)

2176

In [ ]:
item_feature_columns[0].vocabulary_size

2176

# Model Layer Specification

In [ ]:
import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

In [ ]:
from deepctr.feature_column import build_input_features
from deepctr.layers import DNN
from deepctr.layers.utils import NoMask, combined_dnn_input
from tensorflow.python.keras.models import Model

from deepmatch.inputs import input_from_feature_columns, create_embedding_matrix
from deepmatch.layers.core import SampledSoftmaxLayer, EmbeddingIndex, PoolingLayer
from deepmatch.utils import get_item_embedding, l2_normalize

user_dnn_hidden_units=(128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
sampler_config=sampler_config
seed=1024

In [ ]:
if len(item_feature_columns) > 1:
  raise ValueError("Now YoutubeNN only support 1 item feature like item_id")
item_feature_name = item_feature_columns[0].name
item_vocabulary_size = item_feature_columns[0].vocabulary_size

embedding_matrix_dict = create_embedding_matrix(user_feature_columns + item_feature_columns, l2_reg_embedding,
                                                seed=seed)

user_features = build_input_features(user_feature_columns)
user_inputs_list = list(user_features.values())
user_sparse_embedding_list, user_dense_value_list = input_from_feature_columns(user_features, user_feature_columns,
                                                                                l2_reg_embedding, seed=seed,
                                                                                embedding_matrix_dict=embedding_matrix_dict)
user_dnn_input = combined_dnn_input(user_sparse_embedding_list, user_dense_value_list)

item_features = build_input_features(item_feature_columns)
item_inputs_list = list(item_features.values())

user_dnn_out = DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                    dnn_use_bn, output_activation=output_activation, seed=seed)(user_dnn_input)
user_dnn_out = l2_normalize(user_dnn_out)

item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

item_embedding_matrix = embedding_matrix_dict[
    item_feature_name]
item_embedding_weight = NoMask()(item_embedding_matrix(item_index))

pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])

pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
output = SampledSoftmaxLayer(sampler_config._asdict(), temperature)(
    [pooling_item_embedding_weight, user_dnn_out, item_features[item_feature_name]])
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_out)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))

# Model Compile and Training

In [34]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [35]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )


Train on 19816 samples
Epoch 1/20
19816/19816 [==============================] - 2s 91us/sample - loss: 10.5127
Epoch 2/20
19816/19816 [==============================] - 1s 47us/sample - loss: 8.9676
Epoch 3/20
19816/19816 [==============================] - 1s 45us/sample - loss: 8.2192
Epoch 4/20
19816/19816 [==============================] - 1s 45us/sample - loss: 7.2909
Epoch 5/20
19816/19816 [==============================] - 1s 46us/sample - loss: 6.7120
Epoch 6/20
19816/19816 [==============================] - 1s 46us/sample - loss: 6.3746
Epoch 7/20
19816/19816 [==============================] - 1s 46us/sample - loss: 6.2139
Epoch 8/20
19816/19816 [==============================] - 1s 51us/sample - loss: 6.0759
Epoch 9/20
19816/19816 [==============================] - 1s 48us/sample - loss: 5.9982
Epoch 10/20
19816/19816 [==============================] - 1s 46us/sample - loss: 5.8912
Epoch 11/20
19816/19816 [==============================] - 1s 45us/sample - loss: 5.8121
Epoch 

In [ ]:
# model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)
# model.compile(optimizer="adam", loss=sampledsoftmaxloss)
# history = model.fit(train_model_input, train_label,  # train_label,
#                     batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

# Prediction

In [36]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(46, 32)
(2175, 32)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [42]:
user_embs.dot(item_embs.T)

array([[0.6734152 , 0.3835231 , 0.44975585, ..., 0.3656753 , 0.5457875 ,
        0.33492854],
       [0.59980524, 0.5219771 , 0.48886365, ..., 0.394726  , 0.2998712 ,
        0.3419155 ],
       [0.81454813, 0.7190113 , 0.7832263 , ..., 0.650464  , 0.7431598 ,
        0.7058377 ],
       ...,
       [0.64391613, 0.56634134, 0.5598872 , ..., 0.39427054, 0.39551055,
        0.4206981 ],
       [0.5445291 , 0.5637919 , 0.4940445 , ..., 0.5605978 , 0.55514795,
        0.5434837 ],
       [0.60143524, 0.5750293 , 0.3356342 , ..., 0.25642702, 0.37961757,
        0.4583201 ]], dtype=float32)